In [6]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.neighbors import NearestNeighbors
import numpy as np

# Initialize Spotipy with your Spotify credentials
client_id = "fd869ceb8cb643dfbe11a2c9a63efbc7"  # Replace with your client ID
client_secret = "cc0213c6006f44d695abe9c3411dc6b6"  # Replace with your client secret

client_credentials_manager = SpotifyClientCredentials(
    client_id=client_id, client_secret=client_secret
)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [7]:
# Function to search for a song on Spotify and get its metadata
def get_song_features(song_name):
    results = sp.search(q=song_name, limit=1, type='track')
    if not results['tracks']['items']:
        return None

    track = results['tracks']['items'][0]
    song_id = track['id']

    # Get audio features for the song
    features = sp.audio_features(song_id)[0]

    # Use only relevant features (e.g., danceability, energy, valence)
    return [
        features['danceability'],
        features['energy'],
        features['valence'],
    ]

In [8]:
# Fetch a list of popular songs with their features (example)
songs = [
    {"id": 1, "name": "Tum Hi Ho", "genre": "romantic", "artist": "Arijit Singh"},
    {"id": 2, "name": "Badtameez Dil", "genre": "dance", "artist": "Benny Dayal"},
    {"id": 3, "name": "Channa Mereya", "genre": "romantic", "artist": "Arijit Singh"},
    {"id": 4, "name": "Zingaat", "genre": "dance", "artist": "Ajay-Atul"},
    {"id": 5, "name": "Kal Ho Naa Ho", "genre": "romantic", "artist": "Sonu Nigam"},
    {"id": 6, "name": "Ghungroo", "genre": "party", "artist": "Arijit Singh"},
    {"id": 7, "name": "Tera Yaar Hoon Main", "genre": "emotional", "artist": "Arijit Singh"},
    {"id": 8, "name": "Dil Chori", "genre": "party", "artist": "Yo Yo Honey Singh"},
    {"id": 9, "name": "Apna Time Aayega", "genre": "motivational", "artist": "Ranveer Singh"},
    {"id": 10, "name": "Lag Ja Gale", "genre": "classical", "artist": "Lata Mangeshkar"},
    {"id": 11, "name": "Bekhayali", "genre": "emotional", "artist": "Sachet Tandon"},
    {"id": 12, "name": "Senorita", "genre": "dance", "artist": "Farhan, Hrithik, Abhay"},
    {"id": 13, "name": "Kaala Chashma", "genre": "party", "artist": "Badshah"},
    {"id": 14, "name": "Jeene Laga Hoon", "genre": "romantic", "artist": "Atif Aslam"},
    {"id": 15, "name": "Makhna", "genre": "party", "artist": "Yo Yo Honey Singh"},
    {"id": 16, "name": "Tujh Mein Rab Dikhta Hai", "genre": "romantic", "artist": "Roop Kumar Rathod"},
    {"id": 17, "name": "Yeh Jawaani Hai Deewani", "genre": "dance", "artist": "Benny Dayal"},
    {"id": 18, "name": "Phir Le Aya Dil", "genre": "classical", "artist": "Arijit Singh"},
    {"id": 19, "name": "Dil Dhadakne Do", "genre": "party", "artist": "Farhan Akhtar"},
    {"id": 20, "name": "Mere Sapno Ki Rani", "genre": "classic", "artist": "Kishore Kumar"},
]
# Get feature vecors for each song
features = []
song_names = []
for song in songs:
    song_name = song["name"]
    song_features = get_song_features(song_name)
    if song_features:
        features.append(song_features)
        song_names.append(song_name)

# Convert features to numpy array
features = np.array(features)


In [9]:
# Train a KNN model
knn = NearestNeighbors(n_neighbors=2, metric='euclidean')
knn.fit(features)


NearestNeighbors(metric='euclidean', n_neighbors=2)

In [10]:
# Function to recommend similar songs using Spotify features
def recommend_similar_songs(song_name, num_recommendations=2):
    try:
        song_index = song_names.index(song_name)
    except ValueError:
        return f"Song '{song_name}' not found."

    song_feature = np.array(features[song_index]).reshape(1, -1)

    distances, indices = knn.kneighbors(song_feature, n_neighbors=len(song_names))

    recommendations = [
        song_names[i] for i in indices.flatten() if i != song_index
    ][:num_recommendations]

    return recommendations


In [ ]:
# Example usage
if __name__ == "__main__":
    song_name = input("Enter a song name: ")
    recommendations = recommend_similar_songs(song_name)
    print(f"Recommended songs: {recommendations}")
